In [ ]:
# Instala conda (si no está instalado)
!pip install -q condacolab
import condacolab
condacolab.install()

# Instala ViennaRNA desde conda-forge
!conda install -c bioconda viennarna -y

# Verifica instalación
!RNAfold --version
!RNAplot --version


⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:13
🔁 Restarting kernel...
Channels:
 - bioconda
 - conda-forge
Platform: linux-64
Solving environment: | / - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - viennarna


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2025.10.5  |       hbd8a1cb_0         152 KB  conda-forge
    certifi-2025.10.5          |     pyhd8ed1ab_0         156 KB  conda-forge
    conda-24.11.3              |  py311h38be061_0         1.1 MB  conda-forge
    gmp-6.3.0                  |       hac33072_2         449 KB  conda-forge
    gsl-2.7                    |       he838d99_0         3.2 MB  conda-forge
    libblas-3.9.0     

In [ ]:
!apt-get install viennarna -y


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package viennarna


In [ ]:
!sudo apt update
!sudo apt install ghostscript



Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 https://cli.github.com/packages stable InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [5,803 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [6,

In [ ]:
!gs --version


9.55.0


version corregida que pregunta si quiere con filtro de energía o no??:


In [ ]:
# version 2
#!/usr/bin/env python3
"""
gen_vienna_hairpins.py v2.7

Genera M secuencias P(14)-Q(4)-R(14)-S(21), valida con ViennaRNA (RNAfold),
rechaza secuencias que formen más de 1 horquilla o donde Q empareje con P/R,
genera imágenes (RNAplot + gs), guarda resultados en formato Vienna.

Compatible con ViennaRNA 2.7.0

Requisitos:
 - RNAfold y RNAplot en PATH (ViennaRNA 2.7+)
 - Ghostscript 'gs' (para PNG)
"""

import random
import subprocess
import shutil
import os
import sys
import glob
import math
import time
from typing import Set, Tuple, List, Dict

# Parámetros del esquema (fijos según tu especificación)
P = 14
Q = 4
R = 14
S = 21
MIN_LEN = P + Q + R + S  # 53

# Random seed para reproducibilidad (opcional)
random.seed()

# Complemento ADN
COMP = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}

def complement_base(b: str) -> str:
    return COMP.get(b.upper(), 'N')

def complement_reverse(seq: str) -> str:
    return ''.join(complement_base(c) for c in reversed(seq))

def random_seq(length: int) -> str:
    return ''.join(random.choice(['A','C','G','T']) for _ in range(length))

# --- Dot-bracket -> pares (i,j) ---
def pairs_from_dotbracket(db: str) -> Set[Tuple[int,int]]:
    stack = []
    pairs = set()
    for i, ch in enumerate(db):
        if ch == '(':
            stack.append(i)
        elif ch == ')':
            if not stack:
                continue
            j = stack.pop()
            a, b = (j, i) if j < i else (i, j)
            pairs.add((a,b))
    return pairs

# --- Pares diseñados P<->R (índices 0-based) ---
def designed_pairs_indices(P_len: int, Q_len: int, R_len: int) -> Set[Tuple[int,int]]:
    """
    Para i in [0..P_len-1] -> j = P_len + Q_len + (R_len-1 - i)
    """
    pairs = set()
    for i in range(min(P_len, R_len)):
        j = P_len + Q_len + (R_len - 1 - i)
        a, b = (i, j) if i < j else (j, i)
        pairs.add((a,b))
    return pairs

DESIGNED_PAIRS = designed_pairs_indices(P, Q, R)

# --- Ejecutar RNAfold --noPS sobre una secuencia y parsear salida ---
def evaluate_with_rnafold(seq: str, temp: float = 60.0) -> Tuple[str, float]:
    if not shutil.which('RNAfold'):
        raise RuntimeError("RNAfold no encontrado en PATH.")
    try:
        proc = subprocess.run(
            ['RNAfold', '--noPS', f'--temp={temp}'],
            input=seq.encode(),
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            check=True
        )
        out = proc.stdout.decode().strip().splitlines()
        struct_line = out[1].strip()
        parts = struct_line.split()
        struct = parts[0]
        energy = float(parts[-1].strip('()'))
        return struct, energy
    except subprocess.CalledProcessError as e:
        raise RuntimeError(f"Error RNAfold: {e.stderr.decode()}")

# Calcular fracción apareada (θ) explícitamente: θ = 1/(1+e^[ΔG/(RT)])
def fraction_paired(energy_kcal, temp_C):
    """
    Calcula la fracción de moléculas en estado apareado a una temperatura dada.
    """
    R = 0.001987  # kcal/(mol·K)
    T = temp_C + 273.15
    return 1 / (1 + math.exp(energy_kcal / (R * T)))

# --- Generar imagen usando RNAplot (v2.7 compatible) y convertir a PNG ---
def plot_structure(seq: str, struct: str, seq_idx: int) -> Tuple[str, str]:
    """
    Genera archivos seq_{idx}_ss.ps y seq_{idx}.png en carpeta plots/
    Compatible con RNAplot 2.7.0 (sin opciones -o)
    Retorna (ps_path, png_path)
    """
    if not shutil.which('RNAplot'):
        raise RuntimeError("RNAplot no encontrado en PATH.")

    # Verificar que longitudes coincidan
    if len(seq) != len(struct):
        raise RuntimeError(f"Longitudes no coinciden: seq={len(seq)}, struct={len(struct)}")

    # Nombres de archivos finales
    base = f"seq_{seq_idx}"
    ps_path = os.path.join('plots', f"{base}_ss.ps")
    png_path = os.path.join('plots', f"{base}.png")

    input_text = f"{seq}\n{struct}\n"

    # Guardar directorio original
    original_dir = os.getcwd()

    try:
        # Cambiar a directorio plots para que RNAplot genere archivos ahí
        os.chdir('plots')

        # Limpiar archivos PS/EPS antiguos en el directorio
        for old_file in glob.glob('*.ps') + glob.glob('*.eps'):
            try:
                os.remove(old_file)
            except:
                pass

        # RNAplot 2.7 lee stdin y genera rna_ss.ps
        proc = subprocess.run(
            ["RNAplot"],
            input=input_text,
            text=True,
            capture_output=True
        )

        # Verificar si hubo error
        if proc.returncode != 0:
            raise RuntimeError(f"RNAplot falló: {proc.stderr}")

        # Esperar un momento para que el archivo se escriba completamente
        time.sleep(0.1)

        # Buscar archivo .ps o .eps generado (intentar varios nombres comunes)
        possible_names = ['rna_ss.ps', 'rna.ps', 'rna.eps', 'ss.ps']
        generated_ps = None

        for name in possible_names:
            if os.path.exists(name):
                generated_ps = name
                break

        # Si no encontramos ninguno, buscar cualquier .ps/.eps
        if not generated_ps:
            ps_files = glob.glob('*.ps') + glob.glob('*.eps')
            if ps_files:
                generated_ps = ps_files[0]

        if not generated_ps:
            # Debug: listar todos los archivos
            all_files = os.listdir('.')
            raise RuntimeError(f"RNAplot no generó archivo PS/EPS. Archivos en plots/: {all_files}. stderr: {proc.stderr}")

        # Archivo encontrado
        final_ps_name = f"{base}_ss.ps"

        # Renombrar si es necesario
        if generated_ps != final_ps_name:
            if os.path.exists(final_ps_name):
                os.remove(final_ps_name)
            os.rename(generated_ps, final_ps_name)

        ps_path = os.path.join(os.getcwd(), final_ps_name)

        # Convertir a PNG con Ghostscript
        png_created = ''
        if shutil.which('gs'):
            try:
                png_name = f"{base}.png"
                # Ghostscript necesita -dEPSCrop para archivos EPS
                result = subprocess.run([
                    "gs", "-dSAFER", "-dBATCH", "-dNOPAUSE", "-dQUIET",
                    "-dEPSCrop",  # Importante para archivos EPS
                    "-sDEVICE=pngalpha", "-r300",  # Aumentar resolución a 300 DPI
                    f"-sOutputFile={png_name}", final_ps_name
                ], capture_output=True)

                if result.returncode != 0:
                    raise RuntimeError(f"Ghostscript error: {result.stderr.decode()}")

                if os.path.exists(png_name):
                    png_created = os.path.join(os.getcwd(), png_name)
                else:
                    raise RuntimeError(f"PNG no se generó: {png_name}")

            except Exception as e:
                print(f"  ⚠️  Ghostscript falló para seq_{seq_idx}: {e}")
        else:
            print(f"  ⚠️  'gs' no encontrado; solo .ps disponible para seq_{seq_idx}")

        return ps_path, png_created

    except Exception as e:
        raise RuntimeError(f"Error en plot_structure: {e}")
    finally:
        # Volver al directorio original
        os.chdir(original_dir)

# --- Validación de estructura ---
def analyze_structure_validity(struct: str, seq_len: int, P_len: int, Q_len: int, R_len: int) -> Tuple[bool, str]:
    """
    Retorna (valid, message). valid=True si:
      - Los pares coinciden exactamente con DESIGNED_PAIRS
      - Q no forma pares con P o R
    """
    pred_pairs = pairs_from_dotbracket(struct)
    extra_pairs = pred_pairs - DESIGNED_PAIRS
    missing_pairs = DESIGNED_PAIRS - pred_pairs

    q_start = P_len
    q_end = P_len + Q_len - 1
    q_pairs = [p for p in pred_pairs if (q_start <= p[0] <= q_end) or (q_start <= p[1] <= q_end)]

    if extra_pairs:
        return False, f"Pares extra detectados ({len(extra_pairs)})."
    if missing_pairs:
        return False, f"Pares faltantes del stem ({len(missing_pairs)})."
    if q_pairs:
        return False, f"Loop Q presenta emparejamientos no permitidos."
    return True, "Válida"

# --- Generador principal ---
def generate_M_sequences(M: int, max_attempts=10000, energy_filter=None) -> List[Dict]:
    """
    Genera hasta M secuencias válidas.

    Args:
        M: Número de secuencias a generar
        max_attempts: Máximo número de intentos
        energy_filter: Tupla (min, max) para filtrar por energía. None = sin filtro

    Retorna lista de dicts: id, seq, struct, energy, frac, ps, png
    """
    if M <= 0:
        raise ValueError("M debe ser > 0")

    os.makedirs("plots", exist_ok=True)
    accepted = []
    seen = set()
    attempts = 0
    rejected_by_energy = 0

    while len(accepted) < M and attempts < max_attempts:
        attempts += 1

        # Generar secuencia P-Q-R-S
        p_seq = random_seq(P)
        q_seq = random_seq(Q) if Q > 0 else ''
        r_seq = complement_reverse(p_seq)
        s_seq = random_seq(S) if S > 0 else ''
        full_seq = p_seq + q_seq + r_seq + s_seq

        if full_seq in seen:
            continue
        seen.add(full_seq)

        # Evaluar con RNAfold
        try:
            struct, energy = evaluate_with_rnafold(full_seq, temp=60.0)
        except Exception as e:
            print(f"❌ Error RNAfold: {e}")
            continue

        # Validar estructura
        valid, msg = analyze_structure_validity(struct, len(full_seq), P, Q, R)
        if not valid:
            print(f"[RECHAZADA] intento {attempts}: {msg} | {p_seq}... | E={energy:.2f}")
            continue

        # Filtro opcional por energía
        if energy_filter is not None:
            min_e, max_e = energy_filter
            if not (min_e <= energy <= max_e):
                rejected_by_energy += 1
                if rejected_by_energy % 100 == 0:
                    print(f"  [INFO] {rejected_by_energy} secuencias rechazadas por filtro de energía ({min_e} a {max_e} kcal/mol)")
                continue

        # Calcular fracción apareada
        frac = fraction_paired(energy, 60.0)

        # Generar imágenes
        seq_id = len(accepted) + 1
        ps_path, png_path = '', ''
        try:
            ps_path, png_path = plot_structure(full_seq, struct, seq_id)
            print(f"[✓ {len(accepted)+1}/{M}] ID={seq_id} E={energy:.2f} θ={frac:.3f} {full_seq[:12]}... PNG={'✓' if png_path else '✗'}")
        except Exception as e:
            print(f"[✓ {len(accepted)+1}/{M}] ID={seq_id} E={energy:.2f} θ={frac:.3f} {full_seq[:12]}... PNG=✗ (Error: {e})")

        rec = {
            'id': seq_id,
            'seq': full_seq,
            'struct': struct,
            'energy': energy,
            'frac': frac,
            'ps': ps_path,
            'png': png_path
        }
        accepted.append(rec)

    if len(accepted) < M:
        print(f"\n⚠️  Advertencia: {len(accepted)}/{M} secuencias generadas en {attempts} intentos.")
        if energy_filter:
            print(f"    {rejected_by_energy} rechazadas por filtro de energía.")
    else:
        print(f"\n✅ {len(accepted)} secuencias válidas en {attempts} intentos.")
        if energy_filter and rejected_by_energy > 0:
            print(f"   ({rejected_by_energy} rechazadas por filtro de energía)")
    return accepted

# --- Guardar archivos ---
def save_vienna(filename: str, records: List[Dict]):
    with open(filename, 'w') as f:
        for r in records:
            f.write(r['seq'] + '\n')
            f.write(f"{r['struct']} ({r['energy']:.2f})\n")
    print(f"✅ Vienna: {filename}")

def save_csv_summary(filename: str, records: List[Dict]):
    with open(filename, 'w', newline='') as f:
        f.write("id,seq,struct,energy,theta,ps,png\n")
        for r in records:
            f.write(f"{r['id']},{r['seq']},{r['struct']},{r['energy']},{r['frac']:.4f},{r['ps']},{r['png']}\n")
    print(f"✅ CSV: {filename}")

def select_most_stable(records: List[Dict], top_k: int = 5):
    return sorted(records, key=lambda x: x['energy'])[:top_k]

def select_closest_to_equilibrium(records: List[Dict], top_k: int = 5):
    """Selecciona las secuencias con θ más cercano a 0.5 (equilibrio)"""
    return sorted(records, key=lambda x: abs(x['frac'] - 0.5))[:top_k]

# --- CLI principal ---
def main():
    print("=" * 70)
    print("  Generador P(14)-Q(4)-R(14)-S(21) + Validación ViennaRNA 2.7")
    print("=" * 70)

    try:
        M = int(input("\nIngresa M (número de secuencias a generar): ").strip())
    except:
        print("❌ Entrada inválida.")
        return

    # Preguntar si quiere filtro de energía
    use_filter = input("¿Aplicar filtro de energía? (s/n, default=n): ").strip().lower()
    energy_filter = None
    if use_filter == 's':
        try:
            min_e = float(input("  Energía mínima (kcal/mol, ej: -5): ").strip())
            max_e = float(input("  Energía máxima (kcal/mol, ej: 5): ").strip())
            energy_filter = (min_e, max_e)
            print(f"  ✓ Filtro aplicado: {min_e} a {max_e} kcal/mol")
        except:
            print("  ✗ Valores inválidos, sin filtro")

    # Verificar herramientas
    missing = []
    if not shutil.which('RNAfold'): missing.append('RNAfold')
    if not shutil.which('RNAplot'): missing.append('RNAplot')
    if missing:
        print(f"❌ Herramientas faltantes: {', '.join(missing)}")
        return

    if not shutil.which('gs'):
        print("⚠️  Ghostscript (gs) no encontrado. Solo se generarán .ps\n")

    print(f"\n🔬 Generando {M} secuencias válidas (longitud={MIN_LEN})...\n")

    records = generate_M_sequences(M, energy_filter=energy_filter)
    if not records:
        print("❌ No se generaron secuencias válidas.")
        return

    # Guardar resultados
    save_vienna('resultados.vienna', records)
    save_csv_summary('resultados_summary.csv', records)

    # Ranking por estabilidad
    top_stable = select_most_stable(records, top_k=min(5, len(records)))
    print("\n" + "=" * 70)
    print("🏆 TOP 5 ESTRUCTURAS MÁS ESTABLES (energía libre de Gibbs)")
    print("=" * 70)
    for i, t in enumerate(top_stable, 1):
        png_status = "✓" if t['png'] else "✗"
        print(f"{i}. ID={t['id']:3d} | E={t['energy']:6.2f} kcal/mol | θ={t['frac']:.3f} | {t['seq'][:15]}... | PNG={png_status}")

    # Ranking por equilibrio (θ ~ 0.5)
    if len(records) >= 5:
        top_eq = select_closest_to_equilibrium(records, top_k=5)
        print("\n" + "=" * 70)
        print("⚖️  TOP 5 ESTRUCTURAS MÁS CERCANAS AL EQUILIBRIO (θ ≈ 0.5)")
        print("=" * 70)
        for i, t in enumerate(top_eq, 1):
            png_status = "✓" if t['png'] else "✗"
            print(f"{i}. ID={t['id']:3d} | θ={t['frac']:.3f} | E={t['energy']:6.2f} kcal/mol | {t['seq'][:15]}... | PNG={png_status}")

    png_count = sum(1 for r in records if r['png'])
    print(f"\n📊 Resumen: {len(records)} secuencias | {png_count} imágenes PNG generadas")
    print(f"📁 Archivos en: plots/ | resultados.vienna | resultados_summary.csv")
    print("=" * 70)

if __name__ == "__main__":
    main()

  Generador P(14)-Q(4)-R(14)-S(21) + Validación ViennaRNA 2.7

Ingresa M (número de secuencias a generar): 400
¿Aplicar filtro de energía? (s/n, default=n): n

🔬 Generando 400 secuencias válidas (longitud=53)...

[✓ 1/400] ID=1 E=-19.92 θ=1.000 ACACCAGGCGGC... PNG=✓
[✓ 2/400] ID=2 E=-9.96 θ=1.000 AGAGTAAACTTA... PNG=✓
[✓ 3/400] ID=3 E=-14.12 θ=1.000 AATCTACAGACG... PNG=✓
[✓ 4/400] ID=4 E=-11.38 θ=1.000 AGTAAGTCAGTG... PNG=✓
[✓ 5/400] ID=5 E=-17.02 θ=1.000 TCTTACGTCTGG... PNG=✓
[✓ 6/400] ID=6 E=-12.33 θ=1.000 TGAACAACCTAC... PNG=✓
[RECHAZADA] intento 7: Pares extra detectados (3). | GTTCTGCTCTTGAC... | E=-15.35
[✓ 7/400] ID=7 E=-15.43 θ=1.000 CGCGCTGAATAT... PNG=✓
[✓ 8/400] ID=8 E=-10.86 θ=1.000 CTTCCTAATAGC... PNG=✓
[RECHAZADA] intento 10: Pares extra detectados (3). | GGTTTGGAGCTACA... | E=-17.00
[✓ 9/400] ID=9 E=-16.33 θ=1.000 GTGGCTACGGTC... PNG=✓
[✓ 10/400] ID=10 E=-11.14 θ=1.000 AGAATAAGACGT... PNG=✓
[✓ 11/400] ID=11 E=-16.73 θ=1.000 TACACGCAAACC... PNG=✓
[✓ 12/400] ID=12 E=-17.46

In [ ]:
import shutil

# Comprime la carpeta 'plots' en un archivo ZIP
shutil.make_archive("plots", 'zip', "plots")

print("✅ Carpeta 'plots/' comprimida como 'plots.zip'")


✅ Carpeta 'plots/' comprimida como 'plots.zip'


en poo version


In [2]:
!apt update
!apt install -y vienna-rna ghostscript

Hit:1 https://cli.github.com/packages stable InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,085 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,373 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRe

In [3]:
# vienna_generator.py
# -*- coding: utf-8 -*-

import os
import random
import subprocess
import shutil
import glob
import math
import time
from typing import List, Dict, Set, Tuple

# -------------------------------
# Configuration constants
# -------------------------------
P, Q, R, S = 14, 4, 14, 21
MIN_LEN = P + Q + R + S
TEMP_C = 60.0
COMP = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}


# -------------------------------
# Utility class for sequences
# -------------------------------
class SequenceUtils:
    @staticmethod
    def complement_base(b: str) -> str:
        return COMP.get(b.upper(), 'N')

    @staticmethod
    def complement_reverse(seq: str) -> str:
        return ''.join(SequenceUtils.complement_base(c) for c in reversed(seq))

    @staticmethod
    def random_seq(length: int) -> str:
        return ''.join(random.choice('ACGT') for _ in range(length))

    @staticmethod
    def fraction_paired(energy_kcal: float, temp_C: float) -> float:
        R = 0.001987  # kcal/(mol·K)
        T = temp_C + 273.15
        return 1 / (1 + math.exp(energy_kcal / (R * T)))


# -------------------------------
# ViennaRNA execution utilities
# -------------------------------
class ViennaRunner:
    def __init__(self, temp: float = TEMP_C):
        self.temp = temp
        self._check_tools()

    def _check_tools(self):
        missing = [t for t in ["RNAfold", "RNAplot"] if not shutil.which(t)]
        if missing:
            raise EnvironmentError(f"Herramientas faltantes: {', '.join(missing)}")
        if not shutil.which("gs"):
            print("⚠️ Ghostscript no encontrado: se generarán solo archivos .ps")

    def run_rnafold(self, seq: str) -> Tuple[str, float]:
        proc = subprocess.run(
            ["RNAfold", "--noPS", f"--temp={self.temp}"],
            input=seq.encode(),
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            check=True
        )
        lines = proc.stdout.decode().strip().splitlines()
        struct_line = lines[1].strip()
        struct, energy = struct_line.split()[0], float(struct_line.split()[-1].strip("()"))
        return struct, energy

    def plot_structure(self, seq: str, struct: str, seq_id: int) -> Tuple[str, str]:
        os.makedirs("plots", exist_ok=True)
        base = f"seq_{seq_id}"
        ps_path = os.path.join("plots", f"{base}_ss.ps")
        png_path = os.path.join("plots", f"{base}.png")

        input_text = f"{seq}\n{struct}\n"
        original_dir = os.getcwd()
        os.chdir("plots")

        try:
            # limpiar archivos previos
            for old_file in glob.glob("*.ps") + glob.glob("*.eps"):
                os.remove(old_file)
            subprocess.run(["RNAplot"], input=input_text, text=True, capture_output=True)
            time.sleep(0.1)

            ps_file = next((f for f in glob.glob("*.ps") + glob.glob("*.eps")), None)
            if not ps_file:
                raise RuntimeError("RNAplot no generó archivo PS/EPS")

            os.rename(ps_file, f"{base}_ss.ps")
            if shutil.which("gs"):
                subprocess.run([
                    "gs", "-dSAFER", "-dBATCH", "-dNOPAUSE", "-dQUIET", "-dEPSCrop",
                    "-sDEVICE=pngalpha", "-r300",
                    f"-sOutputFile={base}.png", f"{base}_ss.ps"
                ], check=True)
            os.chdir(original_dir)
            return ps_path, png_path if os.path.exists(png_path) else ""
        finally:
            os.chdir(original_dir)


# -------------------------------
# Structure validation
# -------------------------------
class StructureValidator:
    @staticmethod
    def pairs_from_dotbracket(db: str) -> Set[Tuple[int, int]]:
        stack, pairs = [], set()
        for i, ch in enumerate(db):
            if ch == '(':
                stack.append(i)
            elif ch == ')' and stack:
                j = stack.pop()
                pairs.add((min(i, j), max(i, j)))
        return pairs

    @staticmethod
    def designed_pairs_indices(P_len: int, Q_len: int, R_len: int) -> Set[Tuple[int, int]]:
        pairs = set()
        for i in range(min(P_len, R_len)):
            j = P_len + Q_len + (R_len - 1 - i)
            pairs.add((i, j))
        return pairs

    @classmethod
    def analyze_validity(cls, struct: str, P_len: int, Q_len: int, R_len: int) -> Tuple[bool, str]:
        pred_pairs = cls.pairs_from_dotbracket(struct)
        designed = cls.designed_pairs_indices(P_len, Q_len, R_len)
        extra = pred_pairs - designed
        missing = designed - pred_pairs

        q_start, q_end = P_len, P_len + Q_len - 1
        q_pairs = [p for p in pred_pairs if q_start <= p[0] <= q_end or q_start <= p[1] <= q_end]

        if extra: return False, f"Pares extra detectados ({len(extra)})."
        if missing: return False, f"Pares faltantes del stem ({len(missing)})."
        if q_pairs: return False, "Loop Q presenta emparejamientos no permitidos."
        return True, "Válida"


# -------------------------------
# Main sequence generator class
# -------------------------------
class ViennaSequenceGenerator:
    def __init__(self, M: int, energy_filter=None, max_attempts: int = 10000):
        self.M = M
        self.energy_filter = energy_filter
        self.max_attempts = max_attempts
        self.runner = ViennaRunner(temp=TEMP_C)
        self.accepted = []

    def generate(self) -> List[Dict]:
        seen = set()
        attempts = 0
        rejected_energy = 0

        while len(self.accepted) < self.M and attempts < self.max_attempts:
            attempts += 1
            p = SequenceUtils.random_seq(P)
            q = SequenceUtils.random_seq(Q)
            r = SequenceUtils.complement_reverse(p)
            s = SequenceUtils.random_seq(S)
            seq = p + q + r + s

            if seq in seen:
                continue
            seen.add(seq)

            try:
                struct, energy = self.runner.run_rnafold(seq)
            except Exception as e:
                print(f"❌ RNAfold error: {e}")
                continue

            valid, msg = StructureValidator.analyze_validity(struct, P, Q, R)
            if not valid:
                continue

            if self.energy_filter:
                min_e, max_e = self.energy_filter
                if not (min_e <= energy <= max_e):
                    rejected_energy += 1
                    continue

            frac = SequenceUtils.fraction_paired(energy, TEMP_C)
            seq_id = len(self.accepted) + 1
            try:
                ps, png = self.runner.plot_structure(seq, struct, seq_id)
            except Exception:
                ps, png = "", ""

            self.accepted.append({
                "id": seq_id,
                "seq": seq,
                "struct": struct,
                "energy": energy,
                "frac": frac,
                "ps": ps,
                "png": png
            })

        print(f"\n✅ {len(self.accepted)} secuencias válidas generadas ({attempts} intentos).")
        return self.accepted

    # --- Saving and ranking ---
    @staticmethod
    def save_vienna(filename: str, records: List[Dict]):
        with open(filename, "w") as f:
            for r in records:
                f.write(f"{r['seq']}\n{r['struct']} ({r['energy']:.2f})\n")
        print(f"✅ Vienna guardado en {filename}")

    @staticmethod
    def save_csv(filename: str, records: List[Dict]):
        with open(filename, "w") as f:
            f.write("id,seq,struct,energy,theta,ps,png\n")
            for r in records:
                f.write(f"{r['id']},{r['seq']},{r['struct']},{r['energy']:.2f},{r['frac']:.3f},{r['ps']},{r['png']}\n")
        print(f"✅ CSV guardado en {filename}")

    @staticmethod
    def select_most_stable(records, top_k=5):
        return sorted(records, key=lambda x: x["energy"])[:top_k]

    @staticmethod
    def select_closest_to_equilibrium(records, top_k=5):
        return sorted(records, key=lambda x: abs(x["frac"] - 0.5))[:top_k]


# -------------------------------
# CLI wrapper
# -------------------------------
class MainCLI:
    @staticmethod
    def run():
        print("=" * 70)
        print("Vienna Sequence Generator (Refactored OOP Version)")
        print("=" * 70)

        M = int(input("Número de secuencias a generar: "))
        use_filter = input("¿Aplicar filtro de energía? (s/n): ").lower().startswith("s")
        energy_filter = None
        if use_filter:
            min_e = float(input("  Energía mínima (kcal/mol): "))
            max_e = float(input("  Energía máxima (kcal/mol): "))
            energy_filter = (min_e, max_e)

        gen = ViennaSequenceGenerator(M=M, energy_filter=energy_filter)
        records = gen.generate()
        gen.save_vienna("resultados.vienna", records)
        gen.save_csv("resultados_summary.csv", records)

        top_stable = gen.select_most_stable(records)
        print("\n🏆 TOP ESTRUCTURAS MÁS ESTABLES:")
        for t in top_stable:
            print(f"  ID={t['id']} | E={t['energy']:.2f} | θ={t['frac']:.3f}")

        shutil.make_archive("plots", "zip", "plots")
        print("\n📁 Carpeta 'plots/' comprimida en 'plots.zip'")


if __name__ == "__main__":
    MainCLI.run()


Vienna Sequence Generator (Refactored OOP Version)
Número de secuencias a generar: 30
¿Aplicar filtro de energía? (s/n): n

✅ 30 secuencias válidas generadas (38 intentos).
✅ Vienna guardado en resultados.vienna
✅ CSV guardado en resultados_summary.csv

🏆 TOP ESTRUCTURAS MÁS ESTABLES:
  ID=25 | E=-24.76 | θ=1.000
  ID=18 | E=-23.76 | θ=1.000
  ID=9 | E=-20.13 | θ=1.000
  ID=21 | E=-19.96 | θ=1.000
  ID=28 | E=-19.71 | θ=1.000

📁 Carpeta 'plots/' comprimida en 'plots.zip'
